In [1]:
!pip install openai

In [2]:
import openai
openai.api_type = "azure"
openai.api_base = "https://bsaiuks.openai.azure.com/"
openai.api_version = "2023-03-15-preview"
openai.api_key = '25d9022d3d124b9ba44ad4464ce14b3a'


In [45]:
from typing import Dict, Iterable
import json
def stream_jsonl(filename: str) -> Iterable[Dict]:
    """
    Parses each jsonl line and yields it as a dictionary
    """
    if filename.endswith(".gz"):
        with open(filename, "rb") as gzfp:
            with gzip.open(gzfp, "rt") as fp:
                for line in fp:
                    if any(not x.isspace() for x in line):
                        yield json.loads(line)
    else:
        with open(filename, "r") as fp:
            for line in fp:
                if any(not x.isspace() for x in line):
                    yield json.loads(line)


In [22]:
import requests
from evalplus.data import get_human_eval_plus, write_jsonl

problems = {task["task_id"]: task for task in stream_jsonl('HumanEval.jsonl')}

num_samples_per_task = 1
len(list(problems.keys()))
problems

{'HumanEval/0': {'task_id': 'HumanEval/0',
  'prompt': 'from typing import List\n\n\ndef has_close_elements(numbers: List[float], threshold: float) -> bool:\n    """ Check if in given list of numbers, are any two numbers closer to each other than\n    given threshold.\n    >>> has_close_elements([1.0, 2.0, 3.0], 0.5)\n    False\n    >>> has_close_elements([1.0, 2.8, 3.0, 4.0, 5.0, 2.0], 0.3)\n    True\n    """\n',
  'entry_point': 'has_close_elements',
  'canonical_solution': '    for idx, elem in enumerate(numbers):\n        for idx2, elem2 in enumerate(numbers):\n            if idx != idx2:\n                distance = abs(elem - elem2)\n                if distance < threshold:\n                    return True\n\n    return False\n',
  'test': "\n\nMETADATA = {\n    'author': 'jt',\n    'dataset': 'test'\n}\n\n\ndef check(candidate):\n    assert candidate([1.0, 2.0, 3.9, 4.0, 5.0, 2.2], 0.3) == True\n    assert candidate([1.0, 2.0, 3.9, 4.0, 5.0, 2.2], 0.05) == False\n    assert candi

In [38]:
import time
def run(prompt, seed, port = 5000):
    while True:  # Keep trying until we break out
        try:
            result = openai.ChatCompletion.create(
                engine="gpt-35-turbo",
                messages=[
                    {"role": "system", "content": "You are a helpful assistant."},
                    {"role": "user", "content": prompt},
                ],
                temperature=0.0
            )

            response = result['choices'][0]["message"]["content"]
            return response
        except Exception:  # If we hit the rate limit
            print('Phasing')
            time.sleep(1)  # Wait for a second before retrying

In [36]:
def get_function_body(code):
    lines = code.splitlines()
    function_lines = []
    for line in lines:
        # Include lines that start with a whitespace character or are empty
        if line.startswith((' ', '\t')) or line.strip() == '':
            function_lines.append(line)
        # Stop including lines when we hit a line that contains text but does not start with a whitespace character
        elif line.strip() != '' and not line.startswith((' ', '\t')):
            break
    return '\n'.join(function_lines)

def generate_one_completion(prompt_code, seed = -1, port = 5000, long_prompt = False, user_tag = "HUMAN:", assistant_tag = "AI MODEL:", system_prefix = ""):
    print(seed)
    suffix = 'def'+prompt_code.split("def")[1].split("(")[0]+"("
    
    if long_prompt:
        prompt = """%s
%s
Complete the following Python code: 
Notes: respond with the entire complete function definition
do not add any comments, be as concise in your code as possible
use only built-in libraries, assume no additional imports other than those provided (if any)

code:
%s

%s
```python

            %s""" % (system_prefix, user_tag, prompt_code, assistant_tag, suffix)
    else:
        prompt = """```python
%s""" % prompt_code
    
    code_result = run(prompt, seed = seed, port = port)
    result = "\n".join(code_result.split("def")[-1].split("\n")[1:]).split("```")[0]
    result = get_function_body(result)
    print("####", prompt, "####")
    print(result)
    return result

import itertools

def run_benchmark(filename, maxnum=-1, port=5000, long_prompt = False, user_tag = "HUMAN:", assistant_tag = "AI MODEL:", system_prefix = ""):
    iterc = itertools.count()
    problem_keys = list(problems)[:maxnum]
    #print(problem_keys)
    all_samples = []
    n = 1
    for idx, task_id in enumerate(problem_keys):
        # Generate real completions
        for _ in range(num_samples_per_task):
            completion = generate_one_completion(problems[task_id]["prompt"], seed=next(iterc), port=port, long_prompt = long_prompt, user_tag = user_tag, assistant_tag = assistant_tag, system_prefix = system_prefix)
            all_samples.append(dict(task_id=task_id, completion=completion))

        # Create a temporary copy of all_samples, to which we will append 'pass' completions
        temp_samples = all_samples.copy()
        
        # Append 'pass' completions for the rest of the tasks
        for remaining_task_id in list(problems)[idx+1:maxnum] + list(problems)[maxnum:]:
            for _ in range(num_samples_per_task):
                temp_samples.append(dict(task_id=remaining_task_id, completion="    pass"))
        
        # Write all samples to the file, overwriting it completely
        write_jsonl(filename, temp_samples)
        n+=1
    print('n',end='')

In [39]:
run_benchmark("gpt_verylowtemp_long.jsonl", maxnum=-1, port=5000, long_prompt = True, user_tag = "HUMAN:", assistant_tag = "", system_prefix = "")

0
#### 
HUMAN:
Complete the following Python code: 
Notes: respond with the entire complete function definition
do not add any comments, be as concise in your code as possible
use only built-in libraries, assume no additional imports other than those provided (if any)

code:
from typing import List


def has_close_elements(numbers: List[float], threshold: float) -> bool:
    """ Check if in given list of numbers, are any two numbers closer to each other than
    given threshold.
    >>> has_close_elements([1.0, 2.0, 3.0], 0.5)
    False
    >>> has_close_elements([1.0, 2.8, 3.0, 4.0, 5.0, 2.0], 0.3)
    True
    """



```python

            def has_close_elements( ####
    for i in range(len(numbers)):
        for j in range(i+1, len(numbers)):
            if abs(numbers[i] - numbers[j]) < threshold:
                return True
    return False

1
#### 
HUMAN:
Complete the following Python code: 
Notes: respond with the entire complete function definition
do not add any comments, be a

In [127]:
answer = {task["task_id"]: task for task in stream_jsonl(
    'results.jsonl')}
problem_keys = list(problems)[:-1]
num_error = 0
for idx, task_id in enumerate(problem_keys):
    n = problems['HumanEval/101']['test'].find('candidate')
    function = problems['HumanEval/101']["entry_point"]
    l = len(function)
    test_prompt = problems['HumanEval/101']['test']  # [:n] + \
    #function+problems['HumanEval/0']['test'][n+9:]
    test_function = problems['HumanEval/101']['test'][:n] + \
        function+problems['HumanEval/101']['test'][n+9:]
    result_code = problems['HumanEval/101']['prompt'] + \
        answer['HumanEval/101']['completion'] + '\n' + \
        test_prompt + test_function[n-6:n+l+1]
    exec(result_code)
    try:
        exec(result_code)
    except:
        print(task_id, end='')
        num_error += 1
        print('fail')
        continue
print('success rate: ',(164-num_error)/164)


success rate:  1.0


In [102]:
answer

{'HumanEval/0': {'task_id': 'HumanEval/0',
  'completion': '    for i in range(len(numbers)):\n        for j in range(i+1, len(numbers)):\n            if abs(numbers[i] - numbers[j]) < threshold:\n                return True\n    return False\n'},
 'HumanEval/1': {'task_id': 'HumanEval/1',
  'completion': "    stack = []\n    result = []\n    current = ''\n    for char in paren_string.replace(' ', ''):\n        if char == '(':\n            stack.append(char)\n        else:\n            stack.pop()\n        current += char\n        if not stack:\n            result.append(current)\n            current = ''\n    return result"},
 'HumanEval/2': {'task_id': 'HumanEval/2',
  'completion': '    return number % 1'},
 'HumanEval/3': {'task_id': 'HumanEval/3',
  'completion': '    balance = 0\n    for op in operations:\n        balance += op\n        if balance < 0:\n            return True\n    return False\n'},
 'HumanEval/4': {'task_id': 'HumanEval/4',
  'completion': '    mean = sum(number

In [123]:
n = problems['HumanEval/101']['test'].find('candidate')
function = problems['HumanEval/101']["entry_point"]
l = len(function)
test_prompt = problems['HumanEval/101']['test']#[:n] + \
    #function+problems['HumanEval/0']['test'][n+9:]
test_function = problems['HumanEval/101']['test'][:n] + \
        function+problems['HumanEval/101']['test'][n+9:]
result_code = problems['HumanEval/101']['prompt'] + \
    answer['HumanEval/101']['completion'] + '\n'+test_prompt +test_function[n-6:n+l+1]
exec(result_code)



def words_string(s):
    """
    You will be given a string of words separated by commas or spaces. Your task is
    to split the string into words and return an array of the words.
    
    For example:
    words_string("Hi, my name is John") == ["Hi", "my", "name", "is", "John"]
    words_string("One, two, three, four, five, six") == ["One", "two", "three", "four", "five", "six"]
    """
    return s.replace(',', ' ').split()
def check(candidate):

    # Check some simple cases
    assert True, "This prints if this assert fails 1 (good for debugging!)"
    assert candidate("Hi, my name is John") == ["Hi", "my", "name", "is", "John"]
    assert candidate("One, two, three, four, five, six") == ["One", "two", "three", "four", "five", "six"]
    assert candidate("Hi, my name") == ["Hi", "my", "name"]
    assert candidate("One,, two, three, four, five, six,") == ["One", "two", "three", "four", "five", "six"]

    # Check some edge cases that are easy to work out by hand.
    assert True,